# GitHub
Das Projekt ist auf GitHub verfügbar: https://github.com/Oliver-Heer/MSc_WI_DIFA_trading_bot_strategy_analyzer
- Docker & docker-compose um einen lokalen Container mit Python, Anaconda und JupyterLab zu erstellen, ohne diese installieren zu müssen
- Kurzes *Getting Startet* um den Container zu erstellen
- Diese Datei

Sämtliche relevanten Informationen zu diesem Projekt sind jedoch in dieser Datei erklärt und implementiert.

---

## Anmerkung
Die verwendete Bibliothek **Backtrader** weist einen Bug bei der Verwendung von matplotlib in der neusten Version auf. Um diesen zu umgehen, muss Matplotlib in der Version **3.2.2** verwendet werden. Wenn der Fehler
> ImportError Cannot import name 'warnings' from 'matplotlib.dates'

geworfen wird, verwendet ihr eine andere Version von matplotlib

Verwendet: **matplotlib==3.2.2**

---

## Warnung
- Die Resultate und performance Masse sollten jederzeit kritisch hinterfragt werden und **nicht** als finanzieller Rat betrachtet werden
- Nicht jedes performance Mass macht bei jeder Strategie Sinn
- Sämtlicher Code ist selber implementiert und kann **Bugs** oder **Fehler** aufweisen, die **Anwendung erfolgt auf eigene Gefahr**
- Die Analyse berücksichtigen **keine** Zinse oder Dividenden

---

## Einleitung
Es gibt verschiedene Strategien für den Aktien- oder Kryptohandel. Keine Strategie ist perfekt und die Zukunft kann damit auch noch nicht vorausgesagt werden. Jedoch sind die meisten historischen Daten und Verläufe von diesen Wertpapieren online verfügbar und automatisierte Strategien können dadurch vor ihrem Einsatz auf ihre Funktionalität und potenzielle Performance überprüft werden. Dieses Notebook enthält einige einfache Trading-Strategien für Trading-Bots und Analyse-Wrapper um deren Performance zu analysieren.

---

## Ziel
Mit Hilfe diesem JupyterNotebook sollen Trading-Strategien überprüfbar und vergleichbar gemacht werden. Es wird die Python Bibliothek **Backtrader** eingesetzt um die Strategien anhand historischer Daten zu überprüfen und Resultate mit Matplotlib zu plotten. Sämtliche Ticker von **Yahoo Finance** können eingesetzt werden.
- Konfigurierbare Parameter für Strategien wie *Einsatzvermögen*, *Laufzeit* oder *Gebühren*
- Funktionsfähig mit sämtlichen *Ticker* von *Yahoo Finance*
- Plot der Resultate in einem PDF (*analysis.pdf*)
- Einfache performance Masse in einem PDF (*analysis.pdf*)
- Mehrere Strategien pro Ausführung

#### Das Ausführen dieser Datei generiert zwei Outputs:
- **analysis.pdf** mit den performance Massen der Strategie und dazugehörigen Plots
- **backtrader.log** mit zusätzlichen Outputs der Strategien

---

## Struktur

1. Imports und Logging
> Benötigte Bibliotheken und Logging-Konfiguration
1. Globale Konfiguration
> Hier können globale Einstellungen für die Strategien konfiguriert werden
1. Strategien
> Implementierte Strategien, am Anfang jeder Klasse können einige Parameter noch angepasst werden  
> - Golden Cross Strategie  
> - Buy And Hold mit monatlicher Einzahlung Strategie
1. Handler & Ausführung
> Implementation der Strategie-Analyse
1. Plotten der Ergebnisse

---

In [1]:
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# Imports und Logging
# Diese Imports sind für das Ausführen notwendig
# Fehlende Pakete können mittels "pip" installiert werden
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

import backtrader
import yfinance # Benötigt um historische Daten von Yahoo Finance herunterzuladen
import pandas as pd # Die historischen Daten werden als Pandas DataFrame verarbeitet
import datetime # Für Datumsberechnung
import logging # Ausgabe von Logging-Nachrichten in einer Datei (backtrader.log)
import random # Zum Generieren von Zufallszahlen
import math
import numpy as np
import matplotlib.pyplot as plt # Plotting !!Verwendet matplotlib in der Version 3.2.2!!
from matplotlib.backends.backend_pdf import PdfPages # Plotting auf mehreren Seiten
from backtrader import plot # Plotting

# Nebensächliche Konsolenoutputs werden im 'backtrader.log' geloggt.
# Kann ausgeschalten werden, indem level=logging.DEBUG gesetzt wird.
# Wird nach jeder Ausführung wieder überschrieben
logging.basicConfig(level=logging.INFO, filename='backtrader.log', filemode='w', format='%(message)s')

### Konfiguration

In [2]:
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# Globale Konfiguration
# Diese Einstellungen sind für alle Strategien gültig
# Sie werden von der "Cerebro" Komponente von Backtrader benötigt
#
# Wenn noch eine eigene Strategie implementiert wurde, muss diese
# weiter unten bei <Ausführung> eingetragen werden
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

# Initialer Einsatz des Brokers / Initiale Einzahlung
start_cash = 5000

# Prozentuale Gebühren 0.02 = 2% pro Kauf / Verkauf
# Ihr könnt diese auch auf 0 setzen
commission_percentage = 0.02

# Ticker von Yahoo Finance
# https://finance.yahoo.com/most-active
ticker = "AAPL"

# Beginn der historischen Daten, kann auch früher sein
start_date = "2010-01-01"

# Ende der historischen Daten, kann auch später sein
end_date = "2021-12-31"

### Strategien

In [3]:
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# GoldenCross Strategie
# https://en.wikipedia.org/wiki/Moving_average_crossover
#
# Golden Cross = Strategie kauft
# Death Cross = Strategie verkauft
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
class GoldenCross(backtrader.Strategy):
    
    # Die Parameter können angepasst werden
    params = (
        ("fast", 50), # fast moving average
        ("slow", 200), # slow moving average
        ("order_percentage", 0.90) # wieviel vom verfügbaren Vermögen verwendet werden soll pro Handel
    )
    
    def __init__(self):
        logging.info("*********************")
        logging.info("GoldenCross Strategie")
        logging.info("*********************")
        
        # Definition der gleitenden Durchschnitte
        # https://www.backtrader.com/docu/induse/
        self.fast_moving_average = backtrader.indicators.SMA(
            self.data.close,
            period = self.params.fast,
            plotname = str(self.params.fast) + " Tage gleitender Durchschnitt"
        )
        
        self.slow_moving_average = backtrader.indicators.SMA(
            self.data.close,
            period = self.params.slow,
            plotname = str(self.params.slow) + " Tage gleitender Durchschnitt"
        )
        
        # Backtrader Doku: https://www.backtrader.com/docu/indautoref/#crossover
        self.crossover = backtrader.indicators.CrossOver(self.fast_moving_average, self.slow_moving_average)
        
    def next(self):
        # BUY
        if self.position.size == 0: # wir halten keine Anteile
            if self.crossover > 0: # GoldenCross
                funds = (self.params.order_percentage * self.broker.cash)
                self.size = math.floor(funds / self.data.close) # Wir können nur ganze Anteile kaufen => floor Funktion
                
                self.buy(size=self.size)
                logging.info(str(self.datas[0].datetime.date(0)) + " BUY  " + str(self.size) + " mit Preis: " + str(self.data.close[0]))

        # SELL
        if self.position.size > 0: # wir halten Anteile
            if self.crossover < 0:
                logging.info(str(self.datas[0].datetime.date(0)) + " SELL " + str(self.size) + " mit Preis: " + str(self.data.close[0]))
                self.close()

In [4]:
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# Buy and hold Strategie mit monatlichen Einkäufen
# https://en.wikipedia.org/wiki/Buy_and_hold
#
# Diese Strategie kauft monatlich Anteile ein
# Sie verkauft nie
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
class BuyAndHoldMonthly(backtrader.Strategy):

    # Die Parameter können angepasst werden
    params = (
        ("monthly_cash", 500), # Monatliche Einzahlung      
        ("monthdays", [15]), # Wir kaufen immer mitte Monat
        ("order_percentage", 0.95) # wieviel vom verfügbaren Vermögen verwendet werden soll pro Handel
    )

    def __init__(self):
        logging.info("***************************")
        logging.info("BuyAndHoldMonthly Strategie")
        logging.info("***************************")
        # Der Timer triggert den Kauf von Anteilen
        self.add_timer(
            when = backtrader.timer.SESSION_START,
            monthdays = self.params.monthdays,
            monthcarry = True # Falls wir einen Feiertag erwischen, nehmen wir einfach den nächsten Tag
        )

    def start(self):
        self.cash_start = self.broker.get_cash()

    def notify_timer(self, timer, when, *args, **kwargs):
        # Monatliche Einzahlung beim Broker aufaddieren
        self.broker.add_cash(self.params.monthly_cash)
        self.broker.startingcash += self.params.monthly_cash
        
        #BUY
        logging.info(str(self.datas[0].datetime.date(0)) + " BUY  " + str(self.size) + " mit Preis: " + str(self.data.close[0]))
        funds = (self.params.order_percentage * self.broker.cash)
        self.size = math.floor(funds / self.data.close) # Wir können nur ganze Anteile kaufen => floor Funktion
        self.buy(size=self.size)

In [5]:
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# Random Strategie
# Diese Strategie fundiert als Kontrollstrategie
# !! Sie sollte nicht eingesetzt werden !!
# Ist diese Kontrollstrategie besser als eine "seriöse" Strategie,
# sollte die "seriöse" Strategie nochmals überdenkt werden.
#
# Pro Tag wird eine Zahl zwischen 0 bis 100 generiert
# Ist die Zahl 77, werden Anteile gekauft
# Ist die Zahl 0, werden alle Anteile verkauft
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
class RandomStrategy(backtrader.Strategy):
    
    params = (
        ("order_percentage", 0.50), # wieviel vom verfügbaren Vermögen verwendet werden soll pro Handel
    )
    
    def next(self):        
        randomNumber = random.randint(0,100)
        
        if randomNumber == 77:
            # BUY
            funds = (self.params.order_percentage * self.broker.cash)
            self.size = math.floor(funds / self.data.close) # Wir können nur ganze Anteile kaufen => floor Funktion

            self.buy(size=self.size)
            logging.info(str(self.datas[0].datetime.date(0)) + " BUY  " + str(self.size) + " mit Preis: " + str(self.data.close[0]))

        # SELL
        if randomNumber == 0:
            logging.info(str(self.datas[0].datetime.date(0)) + " SELL " + str(self.size) + " mit Preis: " + str(self.data.close[0]))
            self.close()

### Handler & Ausführung

In [6]:
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# CerebroHandler Klasse
# Diese Klasse wrappt die Backtrader funktionalitäten
# Sie erlaubt es auf einfache Weise mehrere Cerebro Instanzen zu handlen
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
class CerebroHandler():
    
    # Initialisiert die Analyzer von Backtrader
    def __init__(self, strategy, ticker, start_date, end_date):
        self.cerebro = backtrader.Cerebro()
        self.set_strategy(strategy)
        self.set_datafeed(ticker, start_date, end_date)
        self.cerebro.addanalyzer(backtrader.analyzers.DrawDown, _name="drawdown")
        self.cerebro.addanalyzer(backtrader.analyzers.AnnualReturn, _name="annualReturn")
        self.cerebro.addanalyzer(backtrader.analyzers.SharpeRatio, _name="sharpe")
    
    # Setter für den initialen Einkauf, default = 5000
    def set_cash(self, cash=5000):
        self.cerebro.broker.set_cash(cash)
    
    # Setter für die prozentualen Gebühren, default = 0
    def set_commission(self, percentage=0):       
        self.cerebro.broker.setcommission(commission=percentage, name=None)
    
    # Setter für den Yahoo Finance Ticker mit Start- und Enddatum
    def set_datafeed(self, ticker, start_date, end_date):
        self.start_date = start_date
        self.end_date = end_date
        self.ticker = ticker
        data = backtrader.feeds.PandasData(dataname=yfinance.download(ticker, start_date, end_date))
        self.cerebro.adddata(data)

    # Setter für die Strategie
    def set_strategy(self, strategy):
        self.strategy_name = strategy.__name__
        self.cerebro.addstrategy(strategy)
        
    # Führt den Handler und die enthaltene Cerebro Instanz aus
    def run(self):
        self.result = self.cerebro.run()[0]

    # Hilfsmethode, um Zahlen auf zwei Dezimalstellen zu runden
    def format_number(self, number):
        if number == None:
            return 0
        return "{:,.2f}".format(number).replace(',', '\'')
    
    # Hilfsmethode, um Tage in Jahre, Monate und Tage aufzuteilen
    def format_date(self, number_of_days):
        years = number_of_days // 365
        months = (number_of_days - years * 365) // 30
        days = (number_of_days - years * 365 - months*30)
        return str(years) + "J " + str(months) + "M " + str(days) + "T"

    # Berechnet die durchschnittliche Rendite der Cerebro Instanz
    # Der Parameter <func> kann hier irgend eine numpy Funktion sein, default wird der Durchschnitt berechnet (np.mean)
    def annual_return(self, dictionary, func=np.mean):
        entries = []
        for key in dictionary:
            entries.append(dictionary[key])
        return self.format_number(func(entries))

    # Aggregiert die Anzahl der Einkäufe und Verkäufe
    # Return: Tupel(#Buy, #Sell)
    def count_orders(self):
        counter_buy = 0
        counter_sell = 0
        for order in self.cerebro.broker.orders:
            # 0 = buy, 1 = sell
            if order.ordtype == 0:
                counter_buy += 1
            else:
                counter_sell += 1
        return (counter_buy, counter_sell)

    # Hilfsmethode, um die Analytikdaten zu formatieren
    def get_analytics(self):    
        orders = self.count_orders()
        annual_return_dict = self.result.analyzers.annualReturn.get_analysis();
        data = [
            ["Einsatz", self.format_number(self.cerebro.broker.startingcash)],
            ["Endwert", self.format_number(self.cerebro.broker.getvalue())],
            ["Gewinn/Verlust", self.format_number(self.cerebro.broker.getvalue() - self.cerebro.broker.startingcash)],
            ["Laufzeit", self.format_date((datetime.datetime.strptime(self.end_date, '%Y-%m-%d') - datetime.datetime.strptime(self.start_date, '%Y-%m-%d')).days)],
            ["Käufe", orders[0]],
            ["Verkäufe", orders[1]],
            ["Max Drawdown", "% " + self.format_number(self.result.analyzers.drawdown.get_analysis().max.drawdown)],
            ["Sharpe Ratio", self.format_number(self.result.analyzers.sharpe.get_analysis()["sharperatio"])],
            ["Ø jährliche Rendite", "Ø " + self.annual_return(annual_return_dict)],
            ["Median jährliche Rendite", "Median " + self.annual_return(annual_return_dict, np.median)],
            ["Min Rendite", "Min " + self.annual_return(annual_return_dict, np.min)],
            ["Max Rendite", "Max " + self.annual_return(annual_return_dict, np.max)],
           ]
        return pd.DataFrame(data, columns = ["Kennzahl", "Wert"])

In [7]:
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# Ausführung
# Hier werden die gewünschten Strategien definiert und ausgeführt
# Wenn noch eine eigene Strategie implementiert wurde, muss diese
# in der <strategies> Liste eingetragen werden
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
strategies = [GoldenCross, BuyAndHoldMonthly, RandomStrategy]
handlers = []

for strategy in strategies:
    strat = CerebroHandler(strategy, ticker, start_date, end_date)
    strat.set_cash(start_cash)
    strat.set_commission(commission_percentage)
    strat.run()
    handlers.append(strat)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


### Plotten

In [8]:
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# Plotten PDF
# Hier wird das generierte PDF zusammengestellt
#
# Matplotlib, Backtrader-Plots und JupyterNotebook vertragen sich
# nicht gut. Solange nur Exceptions und keine Errors geworfen werden,
# sollte das PDF trotdem generiert werden.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
%matplotlib ipympl

a4_portrait = (8.27, 11.69)
a4_landscape = (11.69, 8.27)
plotter = plot.plot.Plot(use="Agg")

with PdfPages("analysis.pdf") as pdf:
    for handler in handlers:       
        fig, ax = plt.subplots(figsize=a4_portrait)
        fig.patch.set_visible(False)
        ax.axis('off')
        
        plt.suptitle(handler.strategy_name + " " + handler.ticker)
        dataframe = handler.get_analytics()
        ytable = ax.table(cellText=dataframe.values, colLabels=dataframe.columns, loc="center", cellLoc="right")        
        ytable.set_fontsize(12)
        ytable.scale(1, 2)
        
        pdf.savefig()
        plt.close()
    
        plt.rcParams["figure.figsize"]=a4_landscape
        fig = plotter.plot(handler.result)
        fig[0].suptitle(handler.strategy_name + " " + handler.ticker)
        pdf.savefig()

# Kann <Javascript Error: IPython is not defined> Fehler werfen, JupyterNotebook mag die Plotts von Backtrader nicht, kann ignoriert werden

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>